### Họ tên: Trần Minh Tiến
### MSSV: 23521587

### Yêu cầu:
- Cắt từ tập ratings.csv khoảng 200 dòng để làm tập test
- Sử dụng phương pháp khuyến nghị dựa trên người dùng để dự đoán rating cho 200 dòng ở trên (Sử dụng lại source code trong Lab2)
- Sử dụng rating vừa dự đoán để tính toán mức độ hiệu quả của hệ khuyến nghị
- Đánh giá theo các độ đo: MSE, RMSE, MAE, NMAE

In [3]:
# Load dữ liệu
import pandas as pd
import numpy as np

ratings = pd.read_csv('movies/ratings.csv')
movies = pd.read_csv('movies/movies.csv')

### Chia tập dữ liệu Train/Test (200 dòng cho test)

In [4]:
# Chia tập dữ liệu: 200 dòng cho test
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(ratings, test_size=200, random_state=42)

print(f"Số lượng ratings trong tập train: {len(X_train)}")
print(f"Số lượng ratings trong tập test: {len(X_test)}")
print("\nMẫu dữ liệu test:")
display(X_test.head())

Số lượng ratings trong tập train: 100636
Số lượng ratings trong tập test: 200

Mẫu dữ liệu test:


,userId,movieId,rating,timestamp
67037,432,77866,4.5,1335139641
42175,288,474,3.0,978465565
93850,599,4351,3.0,1498524542
6187,42,2987,4.0,996262677
12229,75,1610,4.0,1158989841


## Xây dựng ma trận User-Item từ tập train

In [5]:
# Tạo ma trận User-Item (pivot table) từ tập train
ratings_df = X_train.pivot(index='movieId', columns='userId', values='rating').fillna(0)

print(f"Kích thước ma trận User-Item: {ratings_df.shape}")
print(f"Số movies: {ratings_df.shape[0]}, Số users: {ratings_df.shape[1]}")
display(ratings_df.head())

Kích thước ma trận User-Item: (9714, 610)
Số movies: 9714, Số users: 610


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


## Các hàm Collaborative Filtering (từ Lab2)

In [8]:
# Các hàm hỗ trợ từ LAB2
def get_rating(userid, movieid):
    """Lấy rating của user cho movie"""
    try:
        return X_train.loc[(X_train.userId==userid) & (X_train.movieId==movieid), 'rating'].iloc[0]
    except:
        return 0.0

def get_vector(type_vector, id_value):
    """Lấy vector của user hoặc movie"""
    if type_vector == 'user':
        return ratings_df[id_value].values
    elif type_vector == 'movie':
        return ratings_df.loc[id_value].values

def cosine_similarity(vector1, vector2):
    """Tính độ tương đồng cosine giữa 2 vector"""
    both_rated = []
    vector1_filtered = []
    vector2_filtered = []
    
    for i in range(len(vector1)):
        if vector1[i] != 0 and vector2[i] != 0:
            vector1_filtered.append(vector1[i])
            vector2_filtered.append(vector2[i])
            both_rated.append(i)
    
    if len(both_rated) == 0:
        return 0
    
    vector1_filtered = np.array(vector1_filtered)
    vector2_filtered = np.array(vector2_filtered)
    
    dot_product = sum(a * b for a, b in zip(vector1_filtered, vector2_filtered))
    norm_vector1 = np.sqrt(sum(a ** 2 for a in vector1_filtered))
    norm_vector2 = np.sqrt(sum(b ** 2 for b in vector2_filtered))
    
    if norm_vector1 == 0 or norm_vector2 == 0:
        return 0
    
    return dot_product / (norm_vector1 * norm_vector2)

def pearson_similarity(vector1, vector2):
    """Tính hệ số tương quan Pearson giữa 2 vector"""
    both_rated = 0
    vector1_filtered = []
    vector2_filtered = []
    
    for i in range(len(vector1)):
        if vector1[i] != 0 and vector2[i] != 0:
            vector1_filtered.append(vector1[i])
            vector2_filtered.append(vector2[i])
            both_rated += 1
    
    if both_rated == 0:
        return 0
    
    n = len(vector1_filtered)
    mean_vector1 = sum(vector1_filtered) / n
    mean_vector2 = sum(vector2_filtered) / n
    
    numerator = 0
    sum_sq_v1 = 0
    sum_sq_v2 = 0
    
    for i in range(n):
        diff_v1 = vector1_filtered[i] - mean_vector1
        diff_v2 = vector2_filtered[i] - mean_vector2
        numerator += diff_v1 * diff_v2
        sum_sq_v1 += diff_v1 ** 2
        sum_sq_v2 += diff_v2 ** 2
    
    denominator = np.sqrt(sum_sq_v1) * np.sqrt(sum_sq_v2)
    
    if denominator == 0:
        return 0
    
    return numerator / denominator

def most_similar(target_vector, ratings_df, topk, similarity='cosine', axis='user'):
    """Tìm topk láng giềng tương đồng nhất"""
    if axis == 'user':
        ratings_df_transpose = ratings_df.T
    else:
        ratings_df_transpose = ratings_df
    
    ids = ratings_df_transpose.index.tolist()
    scores = []
    
    for i, item_id in enumerate(ids):
        candidate_vector = ratings_df_transpose.iloc[i].values
        
        if np.array_equal(candidate_vector, target_vector):
            continue
        
        if similarity == 'cosine':
            sim = cosine_similarity(target_vector, candidate_vector)
        elif similarity == 'pearson':
            sim = pearson_similarity(target_vector, candidate_vector)
        
        scores.append((sim, item_id))
    
    scores.sort(key=lambda x: x[0], reverse=True)
    return scores[:topk]

In [19]:
import numpy as np
import pandas as pd

def get_recommendation(
    target_id: int,
    topN: int,
    topK: int,
    ratings_df: pd.DataFrame,
    similarity_name: str,
    mode: str # 'user' hoặc 'item'
):
    # Kiểm tra dữ liệu đầu vào
    if mode not in ['user', 'item']:
        raise ValueError("mode phải là 'user' hoặc 'item'")
    if mode == 'user' and target_id not in ratings_df.columns:
        raise ValueError(f"User ID {target_id} không tồn tại trong ratings_df")
    if mode == 'item' and target_id not in ratings_df.index:
        raise ValueError(f"Movie ID {target_id} không tồn tại trong ratings_df")

    # Xác định vector mục tiêu
    target_vector = ratings_df[target_id].values if mode == 'user' else ratings_df.loc[target_id].values

    # Tìm top K tương tự
    similar_entities = most_similar(
        target_vector=target_vector,
        ratings_df=ratings_df,
        topk=topK,
        similarity=similarity_name,
        axis=mode
    )

    # Lấy trung bình rating của entity cần dự đoán
    if mode == 'user':
        target_ratings = ratings_df[target_id]
    else:
        target_ratings = ratings_df.loc[target_id]

    rated_values = target_ratings[target_ratings > 0]
    mean_target = rated_values.mean() if not rated_values.empty else 0

    predictions, sum_similarity = {}, {}

    # Duyệt qua các neighbor
    for sim_score, neighbor_id in similar_entities:
        if neighbor_id == target_id:
            continue

        if mode == 'user':
            # User-based CF
            neighbor_ratings = ratings_df[neighbor_id]
            rated_movies = neighbor_ratings[neighbor_ratings > 0]
            mean_neighbor = rated_movies.mean() if not rated_movies.empty else 0

            for movieid in ratings_df.index:
                if ratings_df.loc[movieid, target_id] == 0:
                    rating_vi = neighbor_ratings[movieid]
                    if rating_vi > 0:
                        diff = rating_vi - mean_neighbor
                        predictions[movieid] = predictions.get(movieid, 0) + sim_score * diff
                        sum_similarity[movieid] = sum_similarity.get(movieid, 0) + sim_score

        else:
            # Item-based CF
            neighbor_ratings = ratings_df.loc[neighbor_id]
            rated_users = neighbor_ratings[neighbor_ratings > 0]
            mean_neighbor = rated_users.mean() if not rated_users.empty else 0

            for userid in ratings_df.columns:
                if ratings_df.loc[target_id, userid] == 0:
                    rating_uj = neighbor_ratings[userid]
                    if rating_uj > 0:
                        diff = rating_uj - mean_neighbor
                        predictions[userid] = predictions.get(userid, 0) + sim_score * diff
                        sum_similarity[userid] = sum_similarity.get(userid, 0) + abs(sim_score)

    # predicted rating
    ranking = []
    for key in predictions:
        if sum_similarity[key] > 0:
            predicted_rating = mean_target + (predictions[key] / sum_similarity[key])
            ranking.append((predicted_rating, key))

    ranking.sort(key=lambda x: x[0], reverse=True)
    return ranking[:topN]


In [20]:
# K = 5
recommendations_k5 = get_recommendation(target_id=20, topN=10, topK=5, ratings_df=ratings_df, similarity_name='cosine', mode='user')

for i, (pred_rating, movieid) in enumerate(recommendations_k5, 1):
    print(f"{i}. Movie ID: {movieid}| Predicted Rating: {pred_rating:.4f}")

1. Movie ID: 112| Predicted Rating: 5.2367
2. Movie ID: 260| Predicted Rating: 5.2367
3. Movie ID: 733| Predicted Rating: 5.2367
4. Movie ID: 737| Predicted Rating: 5.2367
5. Movie ID: 1210| Predicted Rating: 5.2367
6. Movie ID: 1429| Predicted Rating: 5.2367
7. Movie ID: 1476| Predicted Rating: 5.2367
8. Movie ID: 1639| Predicted Rating: 5.2367
9. Movie ID: 110| Predicted Rating: 5.1654
10. Movie ID: 282| Predicted Rating: 5.1654


## Dự đoán rating cho tập test

In [21]:
# Hàm dự đoán rating cho tập test
def get_recommendation_4test(test_data, topK=10, similarity_name='cosine'):
    """
    Dự đoán rating cho tập test bằng User-Based Collaborative Filtering
    """
    X_test_pred = []
    X_test_userid = test_data['userId'].tolist()
    X_test_movieID = test_data['movieId'].tolist()
    
    print(f"------Dự đoán cho tập test với {len(X_test_userid)} điểm dữ liệu-----------")
    
    for i in range(len(X_test_userid)):
        userid = X_test_userid[i]
        movieid = X_test_movieID[i]
        
        # Kiểm tra xem user và movie có trong training data không
        if userid not in ratings_df.columns:
            X_test_pred.append(3.0)  # Rating mặc định cho user mới
            continue
            
        if movieid not in ratings_df.index:
            X_test_pred.append(3.0)  # Rating mặc định cho movie mới
            continue
        
        # Nếu user đã rate movie này trong training, lấy luôn
        if ratings_df.loc[movieid, userid] > 0:
            X_test_pred.append(ratings_df.loc[movieid, userid])
            continue
        
        # Dự đoán rating bằng get_recommendation
        recommendations = get_recommendation(
            target_id=userid,
            topN=1000,
            topK=topK,
            ratings_df=ratings_df,
            similarity_name=similarity_name,
            mode='user'
        )
        
        # Tìm movie trong danh sách khuyến nghị
        predicted_rating = None
        for pred_rating, rec_movieid in recommendations:
            if rec_movieid == movieid:
                predicted_rating = pred_rating
                break
        
        # Nếu tìm thấy thì dùng predicted rating, không thì dùng giá trị mặc định
        if predicted_rating is not None:
            X_test_pred.append(predicted_rating)
        else:
            # Dùng mean rating của user làm fallback
            user_ratings = ratings_df[userid]
            rated_values = user_ratings[user_ratings > 0]
            mean_rating = rated_values.mean() if not rated_values.empty else 3.0
            X_test_pred.append(mean_rating)
        
        if (i + 1) % 20 == 0:
            print(f'----- Đã dự đoán {i + 1}/{len(X_test_userid)} dòng')
    
    return X_test_pred

In [22]:
# Thực hiện dự đoán rating cho 200 dòng test (với K=10)
X_test_pred_rating = get_recommendation_4test(X_test, topK=10)

------Dự đoán cho tập test với 200 điểm dữ liệu-----------
----- Đã dự đoán 20/200 dòng
----- Đã dự đoán 20/200 dòng
----- Đã dự đoán 40/200 dòng
----- Đã dự đoán 40/200 dòng
----- Đã dự đoán 60/200 dòng
----- Đã dự đoán 60/200 dòng
----- Đã dự đoán 80/200 dòng
----- Đã dự đoán 80/200 dòng
----- Đã dự đoán 100/200 dòng
----- Đã dự đoán 100/200 dòng
----- Đã dự đoán 120/200 dòng
----- Đã dự đoán 120/200 dòng
----- Đã dự đoán 140/200 dòng
----- Đã dự đoán 140/200 dòng
----- Đã dự đoán 160/200 dòng
----- Đã dự đoán 160/200 dòng
----- Đã dự đoán 180/200 dòng
----- Đã dự đoán 180/200 dòng
----- Đã dự đoán 200/200 dòng
----- Đã dự đoán 200/200 dòng


## Đánh giá kết quả với các độ đo: MSE, RMSE, MAE, NMAE

In [23]:
# Lấy rating thực tế từ tập test
X_test_true_rating = X_test['rating'].tolist()

print("Số lượng predicted ratings:", len(X_test_pred_rating))
print("Số lượng true ratings:", len(X_test_true_rating))

# Hiển thị một vài kết quả
print("\nMẫu kết quả dự đoán:")
for i in range(min(10, len(X_test_pred_rating))):
    print(f"  Dòng {i+1}: True={X_test_true_rating[i]:.1f}, Predicted={X_test_pred_rating[i]:.2f}")

Số lượng predicted ratings: 200
Số lượng true ratings: 200

Mẫu kết quả dự đoán:
  Dòng 1: True=4.5, Predicted=3.64
  Dòng 2: True=3.0, Predicted=3.15
  Dòng 3: True=3.0, Predicted=2.64
  Dòng 4: True=4.0, Predicted=3.56
  Dòng 5: True=4.0, Predicted=3.22
  Dòng 6: True=4.0, Predicted=3.77
  Dòng 7: True=3.5, Predicted=3.74
  Dòng 8: True=4.5, Predicted=3.04
  Dòng 9: True=0.5, Predicted=3.36
  Dòng 10: True=3.5, Predicted=3.72


In [24]:
X_test_pred_rating

[np.float64(3.642857142857143),
 np.float64(3.146110056925996),
 np.float64(2.6423948220064726),
 np.float64(3.5639269406392695),
 np.float64(3.2205882352941178),
 np.float64(3.773109243697479),
 np.float64(3.7355555555555555),
 np.float64(3.04),
 np.float64(3.3647151898734178),
 np.float64(3.7235576923076925),
 np.float64(2.2428198433420365),
 np.float64(3.7142857142857144),
 np.float64(3.671875),
 np.float64(3.923076923076923),
 np.float64(3.84),
 np.float64(2.664784394250513),
 3.0,
 np.float64(3.861344537815126),
 np.float64(2.897802374939552),
 np.float64(3.234076433121019),
 np.float64(3.5431726907630523),
 3.0,
 np.float64(3.4871794871794872),
 np.float64(4.43),
 np.float64(4.660387448533196),
 np.float64(3.3827433628318584),
 np.float64(3.7395833333333335),
 np.float64(3.5267857142857144),
 np.float64(3.371751237623762),
 np.float64(3.4991749174917492),
 np.float64(3.511782786885246),
 3.0,
 np.float64(3.7052054794520544),
 np.float64(3.8214285714285716),
 np.float64(3.34214618

In [25]:
# Tính các metrics
import math
from sklearn.metrics import mean_squared_error

# 1. MSE (Mean Squared Error)
mse = mean_squared_error(X_test_true_rating, X_test_pred_rating)
print(f'MSE (Mean Squared Error): {mse:.4f}')

# 2. RMSE (Root Mean Squared Error)
rmse = math.sqrt(mse)
print(f'RMSE (Root Mean Squared Error): {rmse:.4f}')

# 3. MAE (Mean Absolute Error)
mae = np.mean(np.abs(np.array(X_test_true_rating) - np.array(X_test_pred_rating)))
print(f'MAE (Mean Absolute Error): {mae:.4f}')

# 4. NMAE (Normalized Mean Absolute Error)
# NMAE = MAE / (max_rating - min_rating)
rating_range = max(X_test_true_rating) - min(X_test_true_rating)
nmae = mae / rating_range if rating_range > 0 else 0
print(f'NMAE (Normalized Mean Absolute Error): {nmae:.4f}')

MSE (Mean Squared Error): 1.0682
RMSE (Root Mean Squared Error): 1.0335
MAE (Mean Absolute Error): 0.8059
NMAE (Normalized Mean Absolute Error): 0.1791
